## Accessing Mistral 7B

## Trainning Mistral 7B - Conversational with MetaMathQA_395k (Lấy dữ liệu 100k thứ II)

### Bước 0: Cài thư viện cần thiết

In [ ]:
%%capture
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install unsloth vllm
# !pip install --upgrade transformers==4.52.3

### Bước 0: Đăng nhập HuggingFace để lấy mô hình

In [ ]:
!huggingface-cli login --token $secret_hf

usage: huggingface-cli <command> [<args>] login [-h] [--token TOKEN] [--add-to-git-credential]
huggingface-cli <command> [<args>] login: error: argument --token: expected one argument


In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

In [ ]:
from huggingface_hub import login
login(token=hf_token, new_session=False)

### Bước 1: Load mô hình Mistral 4bit từ Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Chiều dài tối đa 1 đoạn văn bản
dtype = None           # Tự động chọn (float16, bfloat16) theo GPU
load_in_4bit = True    # Dùng mô hình lượng tử hóa 4-bit
lora_rank = 32

# Load model đã quant hóa 4bit + tự động chia lên GPU phù hợp
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-29 05:19:33.561873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751174373.747542      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751174373.803709      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-29 05:19:51 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-29 05:19:51 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.6.8: Fast Mistral patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/mistral-7b-instruct-v0.3-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

...

INFO 06-29 05:22:08 [model_runner.py:1592] Graph capturing finished in 49 secs, took 0.53 GiB
INFO 06-29 05:22:08 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 66.04 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'q_norm', 'k_norm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'q_norm', 'k_norm', 'pre_feedforward_layernorm']


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

### Bước 2: Áp dụng LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_rank,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.6.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


###  Bước 3: Gán Chat Template kiểu ChatML. Load và xử lý dữ liệu MetaMathQA

### Train/test split

In [ ]:
from datasets import load_dataset

dataset = load_dataset("meta-math/MetaMathQA", split="train")
dataset = dataset.select(range(100000, 200000))  # Chỉ lấy 100k mẫu từ 100000 - 200000
print(len(dataset))

README.md: 0.00B [00:00, ?B/s]

MetaMathQA-395K.json:   0%|          | 0.00/396M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/395000 [00:00<?, ? examples/s]

100000


In [ ]:
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

### Format dạng hội thoại với system instruction

In [ ]:
from unsloth.chat_templates import get_chat_template

# Gán template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",  # hoặc "mistral", "unsloth"
    map_eos_token=True,
)

# System Instruction
system_instruction_fixed = """Below is an instruction that describes a mathematical task.
Write a response that thoroughly solves the given problem.
Before solving, develop a clear step-by-step chain of reasoning to ensure accuracy and logical coherence.

### Instruction:
You are a mathematics expert with advanced knowledge in mathematical reasoning, problem-solving, and proof techniques. You think outloud and consider various aspects before giving any concrete answers."""

def formatting_prompts_func_conversational_structured(examples):
    queries = examples["query"]
    responses = examples["response"]
    texts = []

    for q, r in zip(queries, responses):
        messages = [
            {"role": "system", "content": system_instruction_fixed},
            {"role": "user", "content": q},
            {"role": "assistant", "content": r},
        ]
        formatted = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(formatted)

    return {"text": texts}


Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
train_dataset = train_dataset.map(formatting_prompts_func_conversational_structured, batched=True)
eval_dataset = eval_dataset.map(formatting_prompts_func_conversational_structured, batched=True)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

### Tokenize train and test dataset

In [ ]:
# # Tokenize
# def tokenize_func(example):
#     tokenized = tokenizer(
#         example["text"],
#         truncation=True,
#         max_length=2048,
#         padding="max_length",
#     )
#     tokenized["labels"] = tokenized["input_ids"].copy()
#     return tokenized


In [ ]:
# tokenized_train = train_dataset.map(tokenize_func, batched=False)
# tokenized_eval = eval_dataset.map(tokenize_func, batched=False)

### Bước 5: Huấn luyện bằng `SFTTrainer`

In [ ]:
from trl import SFTConfig, SFTTrainer

# Khởi tạo Trainer để huấn luyện mô hình với các thiết lập đã tối ưu cho toán học
trainer = SFTTrainer(
    model = model,                          # Mô hình đã được chuẩn bị (có thể LoRA hoặc fine-tuned)
    tokenizer = tokenizer,                  # Tokenizer phù hợp với mô hình
    train_dataset = train_dataset,        # Dữ liệu đã được token hóa cho huấn luyện
    eval_dataset = eval_dataset,          # Dữ liệu để đánh giá trong quá trình train
    dataset_text_field = "text",            # Trường chứa chuỗi đầu vào
    max_seq_length = 2048,                  # Độ dài tối đa mỗi chuỗi, phù hợp với Mistral
    dataset_num_proc = 2,                   # Số tiến trình song song để xử lý dữ liệu
    packing = False,                        # Không gộp nhiều mẫu ngắn lại – tốt hơn cho toán học vì mẫu thường dài
    args = SFTConfig(
        output_dir = "outputs",             # Thư mục để lưu checkpoint và log
        per_device_train_batch_size = 2,    # Batch size mỗi GPU – tăng nếu đủ VRAM
        per_device_eval_batch_size = 2,     # Batch size khi đánh giá – không ảnh hưởng tốc độ train
        gradient_accumulation_steps = 4,    # Số bước tích lũy gradient – tăng nếu VRAM hạn chế
        warmup_steps = 60,                  # Số bước đầu warmup learning rate (5% ~ 1000 steps)
        max_steps = 1200,                   # Tổng số bước huấn luyện (bạn có thể giảm nếu cần)
        learning_rate = 2e-4,               # Learning rate – thường dùng 1e-4 cho LoRA
        optim = "adamw_8bit",               # Optimizer nhẹ, phù hợp với mô hình 4-bit
        weight_decay = 0.01,                # Tránh overfitting nhẹ
        lr_scheduler_type = "cosine",       # Tăng giảm learning rate mượt mà hơn “linear”
        logging_steps = 10,                 # In log sau mỗi 10 bước
        report_to = "none",                 # Nếu muốn theo dõi bằng wandb thì đổi thành "wandb"
        seed = 3407,                        # Đảm bảo kết quả reproducible nếu cần
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/80000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
# Bắt đầu quá trình huấn luyện
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 80,000 | Num Epochs = 1 | Total steps = 1,200
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080/7,000,000,000 (1.20% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.394700
20,0.707300
30,0.452300
40,0.388600
50,0.379300
60,0.357400
70,0.351700
80,0.360100
90,0.357700
100,0.348700


TrainOutput(global_step=1200, training_loss=0.3185234198967616, metrics={'train_runtime': 25627.2156, 'train_samples_per_second': 0.749, 'train_steps_per_second': 0.047, 'total_flos': 2.1799128472682496e+17, 'train_loss': 0.3185234198967616})

### Bước 7: Inference sau khi huấn luyện

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer
import torch

# Kích hoạt chế độ inference nhanh
FastLanguageModel.for_inference(model)

# Gắn lại chat template chuẩn chatml (hoặc mistral nếu fine-tune theo kiểu đó)
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    map_eos_token=True,
)

# Hàm hỏi đáp inference
def chat(question, max_new_tokens=512, stream=False):
    messages = [{"from": "human", "value": question}]

    # Tạo prompt theo chuẩn chat
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    # Tùy chọn stream ra màn hình trực tiếp
    if stream:
        streamer = TextStreamer(tokenizer)
        _ = model.generate(
            input_ids=inputs,
            streamer=streamer,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            top_k=1,
            repetition_penalty=1.1,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    else:
        # Sinh ra kết quả hoàn chỉnh
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            top_k=1,
            repetition_penalty=1.1,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,
        )

        # Giải mã kết quả
        result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # Cắt phần prompt nếu cần
        response = result.split(question.strip())[-1].strip()
        return response


Unsloth: Will map <|im_end|> to EOS = <|im_end|>.


In [ ]:
response = chat("If $x - y = X and $x + y = 12$, what is the value of $x$? If we know the answer to the above question is 9, what is the value of unknown variable X?")
print(response) # Đúng

<|im_start|>assistant
We are given that $x - y = X$ and $x + y = 12$.
To find the value of $x$, we can solve this system of equations.
Adding the two equations together, we get:
$(x - y) + (x + y) = X + 12$
Simplifying, we have:
$2x = X + 12$
Dividing both sides by 2, we find:
$x = \frac{X + 12}{2}$
Given that the value of $x$ is 9, we can substitute it into the equation:
$9 = \frac{X + 12}{2}$
Multiplying both sides of the equation by 2 to eliminate the fraction, we get:
$18 = X + 12$
Subtracting 12 from both sides of the equation, we find:
$6 = X$
The value of X is 6.
The answer is: 6


In [ ]:
chat("A radio show plays for 3 hours a day. They split their show into talking segments, ad breaks and songs. Talking segments last 10 minutes each, ad breaks last 5 minutes each and songs are played throughout the rest of the show. If the radio show includes 3 talking segments and x ad breaks in today’s show, how long, in minutes, does the show play songs? If we know the answer to the above question is 125, what is the value of unknown variable x?", stream=True)
# Đúng

<|im_start|>user
A radio show plays for 3 hours a day. They split their show into talking segments, ad breaks and songs. Talking segments last 10 minutes each, ad breaks last 5 minutes each and songs are played throughout the rest of the show. If the radio show includes 3 talking segments and x ad breaks in today’s show, how long, in minutes, does the show play songs? If we know the answer to the above question is 125, what is the value of unknown variable x?<|im_end|>
<|im_start|>assistant
The radio show plays for 3 hours a day, which is equal to 3 * 60 = 180 minutes.
Talking segments last 10 minutes each, so 3 talking segments will last 3 * 10 = 30 minutes.
Ad breaks last 5 minutes each, so x ad breaks will last x * 5 minutes.
The remaining time is the total duration minus the time spent on talking segments and ad breaks: 180 - 30 - (x * 5) minutes.
We are given that the show plays songs for 125 minutes, so we can write: 180 - 30 - (x * 5) = 125.
Simplifying the left side, we get: 15

In [ ]:
response = chat("In a charity race to raise money for hurricane victims, thirty students participated. Ten of them raised $20 each, while the remaining students raised $30 each. What is the total amount of money raised by the students in the race?")
print(response)
# Đúng

<|im_start|>assistant
The ten students who raised $20 each contributed a total of 10 * $20 = $200.
The remaining twenty students who raised $30 each contributed a total of 20 * $30 = $600.
Therefore, the total amount of money raised by the students in the race is $200 + $600 = $800.
#### 800
The answer is: 800


In [ ]:
chat("Which two-digit positive integer is one more than a multiple of 2, 3, 4, 5, and 6?", stream=True)
# Sai, đáp là: 61 (Giải đúng đc 1 phần)

<|im_start|>user
Which two-digit positive integer is one more than a multiple of 2, 3, 4, 5, and 6?<|im_end|>
<|im_start|>assistant
The least common multiple (LCM) of 2, 3, 4, 5, and 6 is 60.
To find the two-digit positive integer that is one more than a multiple of 60, we add 1 to each multiple of 60 until we get a two-digit number.
The first multiple of 60 is 60, so we add 1 to get 61.
The next multiple of 60 is 120, so we add 1 to get 121.
The next multiple of 60 is 180, so we add 1 to get 181.
The next multiple of 60 is 240, so we add 1 to get 241.
The next multiple of 60 is 300, so we add 1 to get 301.
Therefore, the two-digit positive integer that is one more than a multiple of 2, 3, 4, 5, and 6 is $\boxed{301}$.The answer is: 301<|im_end|>


### Bước 8: Lưu mô hình

#### Tải mô hình

In [ ]:
model.save_pretrained("mistral-metamathqa-lora-100k-v1")
tokenizer.save_pretrained("mistral-metamathqa-lora-100k-v1")

('mistral-metamathqa-lora-100k-v1/tokenizer_config.json',
 'mistral-metamathqa-lora-100k-v1/special_tokens_map.json',
 'mistral-metamathqa-lora-100k-v1/tokenizer.json')

In [ ]:
import shutil

shutil.make_archive(
    base_name="/kaggle/working/mistral-metamathqa-lora-100k-v1",  # Tên file zip (không cần .zip ở đây)
    format="zip",  # Có thể là: 'zip', 'tar', 'gztar', 'bztar', 'xztar'
    root_dir="/kaggle/working/mistral-metamathqa-lora-100k-v1"
)

'/kaggle/working/mistral-metamathqa-lora-100k-v1.zip'

### Load mô hình lên HuggingFace

In [ ]:
from huggingface_hub import create_repo

create_repo("mistral-metamathqa-lora-100k-v1", private=False)  # hoặc private=False nếu muốn công khai

RepoUrl('https://huggingface.co/hahuy2004/mistral-metamathqa-lora-100k-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='hahuy2004/mistral-metamathqa-lora-100k-v1')

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="/kaggle/working/mistral-metamathqa-lora-100k-v1",
    repo_id="hahuy2004/mistral-metamathqa-lora-100k-v1",  # sửa lại đúng username Hugging Face của bạn
    repo_type="model"
)


adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hahuy2004/mistral-metamathqa-lora-100k-v1/commit/955deeeaa7cb90a096a5b5c36c5c833a7bbe98f1', commit_message='Upload folder using huggingface_hub', commit_description='', oid='955deeeaa7cb90a096a5b5c36c5c833a7bbe98f1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hahuy2004/mistral-metamathqa-lora-100k-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='hahuy2004/mistral-metamathqa-lora-100k-v1'), pr_revision=None, pr_num=None)